## Streaming in LangGraph
In LLMs, streaming means the model starts sending tokens (words) as soon as they're generated, instead of waiting for the entire response to be ready before returning it.

**Why Streaming**
1. Faster response time - low drop-off rates
2. Mimics human like conversation(Builds trust, feels alive and keeps the user engaged)
3. Important for Multi-modal UIs
4. Better UX for long output such as code
5. You can cancel midway saving tokens
6. You can interleave Ul updates, e.g., show "thinking...", show tool results

Doc :  https://docs.langchain.com/oss/python/langgraph/streaming

Taking the exact code as it is which was developed in last notebook `LangGraph/12 Building Chatbot with UI in LangGraph & Streamlit.ipynb`


and for streaming we need to call `.stream` instead `.invoke`

In [2]:
import os
import subprocess
import time

# --- CONFIGURATION ---
target_file = "/content/streamlit_frontend_stream.py"
# ---------------------

if not os.path.exists(target_file):
    print(f"❌ ERROR: '{target_file}' not found. Please upload it first.")
else:
    # 1. Install & Setup
    subprocess.run(["pip", "install", "-q", "streamlit"], check=True)
    if not os.path.exists("cloudflared-linux-amd64"):
        subprocess.run(["wget", "-q", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"], check=True)
        subprocess.run(["chmod", "+x", "cloudflared-linux-amd64"], check=True)

    # 2. Cleanup old processes
    subprocess.run(["pkill", "-f", "streamlit"])
    subprocess.run(["pkill", "-f", "cloudflared"])
    time.sleep(2)

    # 3. Run Streamlit
    print(f"Starting Streamlit with {target_file}...")
    subprocess.Popen(["streamlit", "run", target_file, "--server.port", "8501"])

    # 4. Run Tunnel
    print("Starting Cloudflare Tunnel...")
    with open("cloudflared.log", "w") as log_file:
        subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"], stdout=log_file, stderr=log_file)

    # 5. Wait & Extract (Increased wait time to 10s)
    print("Waiting 10 seconds for the tunnel to initialize...")
    time.sleep(10)

    try:
        # Fixed the regex warning by using r'' for raw string
        cmd = r"grep -o 'https://.*\.trycloudflare.com' cloudflared.log | head -n 1"
        url_match = subprocess.check_output(cmd, shell=True).decode("utf-8").strip()

        if url_match:
            print("\n" + "="*40)
            print(f"🚀 Your App URL: {url_match}")
            print("="*40)
        else:
            print("⚠️ URL not found yet. Check logs manually.")
    except Exception as e:
        print(f"Error: {e}")

Starting Streamlit with /content/streamlit_frontend_stream.py...
Starting Cloudflare Tunnel...
Waiting 10 seconds for the tunnel to initialize...

🚀 Your App URL: https://hands-reduced-tiger-guided.trycloudflare.com


langgraph_backend.py
- -----------------

```
# Authentication
import google.auth
credentials, project_id = google.auth.default()

# Import required libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from typing import TypedDict, Literal, Annotated

# Initialize the Vertex AI Model with the dynamic projectID
model = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    project=project_id,
    location="global",
    streaming=True
)

class ChatState(TypedDict):
    messages : Annotated[list[BaseMessage], add_messages]


def chat_node(state: ChatState) -> ChatState:
    # take user query from state
    messages = state['messages']

    # pass it to LLM
    response = model.invoke(messages)

    # store the received response in state
    return{'messages':[response]}


# check pointer
checkpointer = MemorySaver()

# graph
graph = StateGraph(ChatState)

# node
graph.add_node('chat_node',chat_node)

# edges
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

# compile
chatbot = graph.compile(checkpointer=checkpointer)

```

streamlit_frontend_stream
- ------------------------

```
import streamlit as st
import time
from langgraph_backend import chatbot
from langchain_core.messages import HumanMessage, AIMessageChunk

# 1. Initialize History
if 'message_history' not in st.session_state:
    st.session_state['message_history'] = []

# 2. Display History
for message in st.session_state['message_history']:
    with st.chat_message(message['role']):
        st.markdown(message['content'])

# 3. Define the Generator Function (Updated for "Fake" Streaming)
def get_response_generator(user_input, config):
    """
    1. Gets the full response from backend (because backend uses invoke).
    2. Breaks it into words to simulate a 'token-by-token' stream.
    """
    # This call waits for the full response because backend is 'invoke'
    response_stream = chatbot.stream(
        {'messages': [HumanMessage(content=user_input)]},
        config=config,
        stream_mode="messages"
    )

    for chunk, metadata in response_stream:
        if isinstance(chunk, AIMessageChunk) or (hasattr(chunk, 'type') and chunk.type == 'ai'):
            content = chunk.content
            
            # Handling Gemini 3 List Structure
            full_text = ""
            if isinstance(content, list):
                for block in content:
                    if isinstance(block, dict) and 'text' in block:
                        full_text += block['text']
            elif isinstance(content, str):
                full_text = content
            
            # --- SIMULATE STREAMING ---
            # Split by words and yield them one by one with a tiny delay
            # This makes it look like it's typing!
            for word in full_text.split(" "):
                yield word + " "
                time.sleep(0.05) # Adjust speed here (0.02 is fast, 0.1 is slow)

# 4. Chat Input Logic
user_input = st.chat_input('Type here')

if user_input:
    st.session_state['message_history'].append({'role': 'user', 'content': user_input})
    with st.chat_message('user'):
        st.markdown(user_input)

    with st.chat_message('assistant'):
        run_config = {"configurable": {"thread_id": "1"}}
        
        # st.write_stream consumes our generator
        full_response = st.write_stream(
            get_response_generator(user_input, run_config)
        )
    
    st.session_state['message_history'].append({'role': 'assistant', 'content': full_response})
```